## 0. Libraries, Importing Primary Dataset

In [ ]:
# Phases 1-3 package imports

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import scipy
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense, Dropout

import h5py

In [ ]:
# Importing X dataset with actual image data

X = pd.read_csv("X.csv", sep=' ', header=None, dtype=float)
X = X.values

In [ ]:
# Importing Bush, Williams label datasets

y = pd.read_csv("y_bush_vs_others.csv", header=None)
y_bush = y.values.ravel()
# Number of positive Bush instances: np.sum(y_bush) = 530
y = pd.read_csv("y_williams_vs_others.csv", header=None)
y_williams = y.values.ravel() 
# Number of positive Williams instances: np.sum(y_williams) = 52

In [ ]:
# All datasets contain 13233 items/labels, should return True

X.shape[0] == y_bush.shape[0] == y_williams.shape[0]

## 1. KNN and SVMs (Phase 1)

### 1.1 Training and Testing KNN Classifier

In [ ]:
# Creating KNN classifiers which resulted in highest mean F1

knn_bush = KNeighborsClassifier(n_neighbors=1)
knn_williams = KNeighborsClassifier(n_neighbors=1)

In [ ]:
# Using cross-validation to train using KNN

cv_results_bush = cross_validate(knn_bush, X, y_bush, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=7000), 
                            scoring=('precision', 'recall', 'f1'), return_train_score=False, n_jobs=-1)
cv_results_bush

In [ ]:
cv_results_williams = cross_validate(knn_williams, X, y_williams, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=7000), 
                            scoring=('precision', 'recall', 'f1'), return_train_score=False, n_jobs=-1)
cv_results_williams

### 1.2 Training and Testing SVC Classifier

In [ ]:
# Creating SVC classifiers which resulted in highest mean F1

svc_model_bush = SVC(C=10000, gamma=0.0001, kernel='rbf')
svc_model_williams = SVC(C=0.04, kernel='linear')

In [ ]:
# Using cross-validation to train using SVC

cv_results_bush = cross_validate(svc_model_bush, X, y_bush, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=7000), 
                        scoring=('precision', 'recall', 'f1'), return_train_score=False, n_jobs=-1)
cv_results_bush

In [ ]:
cv_results_williams = cross_validate(svc_model_williams, X, y_williams, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=7000), 
                        scoring=('precision', 'recall', 'f1'), return_train_score=False, n_jobs=-1)
cv_results_williams

## 2. Applying PCA for Dimensionality Reduction (Phase 2)

### 2.1 Transform Data

In [ ]:
pca_3072 = PCA(n_components=3072)
pca_2048 = PCA(n_components=2048)
pca_256 = PCA(n_components=256)
pca_64 = PCA(n_components=64)

In [ ]:
pca_3072.fit(X)
X_pca_3072 = pca_3072.transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca_3072.shape)

In [ ]:
pca_2048.fit(X)
X_pca_2048 = pca_2048.transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca_2048.shape)

In [ ]:
pca_256.fit(X)
X_pca_256 = pca_256.transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca_256.shape)

In [ ]:
pca_64.fit(X)
X_pca_64 = pca_64.transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca_64.shape)

### 2.2 Applying PCA to KNN

In [ ]:
cv_results_bush = cross_validate(knn_bush, X_pca_64, y_bush, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=7000), 
                            scoring=('precision', 'recall', 'f1'), return_train_score=False, n_jobs=-1)
print(cv_results_bush,"\n")

In [ ]:
cv_results_williams = cross_validate(knn_williams, X_pca_256, y_williams, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=7000), 
                            scoring=('precision', 'recall', 'f1'), return_train_score=False, n_jobs=-1)
print(cv_results_williams,"\n")

### 2.3 Applying PCA to SVC

In [ ]:
# 3072 components on original Bush model
cv_results_bush = cross_validate(svc_model_bush, X_pca_3072, y_bush, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=7000), 
                        scoring=('precision', 'recall', 'f1'), return_train_score=False, n_jobs=-1)
print(cv_results_bush,"\n")

In [ ]:
# 2048 components on original Williams model
cv_results_bush = cross_validate(svc_model_williams, X_pca_2048, y_williams, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=7000), 
                        scoring=('precision', 'recall', 'f1'), return_train_score=False, n_jobs=-1)
print(cv_results_bush,"\n")

In [ ]:
# New better Williams model
svc_model_williams_2 = SVC(C=0.125, kernel='linear')

# 2048 components on better Williams model
cv_results_bush = cross_validate(svc_model_williams_2, X_pca_2048, y_williams, cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=7000), 
                        scoring=('precision', 'recall', 'f1'), return_train_score=False, n_jobs=-1)
print(cv_results_bush,"\n")

## 3. CNNs (Phase 3)

### 3.1 Creating Test/Train Split For Datasets

In [ ]:
# For Bush
X_train_bush, X_test_bush, y_train_bush, y_test_bush = train_test_split(
    X, y_bush, test_size=1./3, random_state=4000, shuffle=True, stratify=y_bush)

In [ ]:
# For Williams
X_train_williams, X_test_williams, y_train_williams, y_test_williams = train_test_split(
    X, y_williams, test_size=1./3, random_state=4000, shuffle=True, stratify=y_williams)

In [ ]:
# Reshaping Bush data points to 64x64 for use in CNN
X_train_bush = X_train_bush.reshape(X_train_bush.shape[0],64,64,1) # last param: 1
X_test_bush = X_test_bush.reshape(X_test_bush.shape[0],64,64,1) # last param: 1

In [ ]:
# Reshaping Williams data points to 64x64 for use in CNN
X_train_williams = X_train_williams.reshape(X_train_williams.shape[0],64,64,1) # last param: 1
X_test_williams = X_test_williams.reshape(X_test_williams.shape[0],64,64,1) # last param: 1

In [ ]:
# Visualizing an image (Pete Sampras!)
plt.imshow(X_test_bush[504].reshape(64,64))

In [ ]:
# Visualizing an image (George Bush!)
plt.imshow(X_test_bush[517].reshape(64,64))

In [ ]:
# Function to compute F1
def calc_f1(CNN_predicts, y_vals):
    # Create confusion matrix
    TP_b = 0; TN_b = 0; FP_b = 0; FN_b = 0
    for i in range(0,len(CNN_predicts)):
        if(y_vals[i] == 1 and CNN_predicts[i] == 1):
            TP_b += 1
        if(y_vals[i] == 0 and CNN_predicts[i] == 1):
            FP_b += 1
        if(y_vals[i] == 1 and CNN_predicts[i] == 0):
            FN_b += 1
        else:
            TN_b += 1
    
    print(TP_b,TN_b,FP_b,FN_b)

    b_accuracy = (TP_b+TN_b)/(TP_b+TN_b+FP_b+FN_b)
    b_precision = (TP_b)/(TP_b+FP_b)
    b_recall = (TP_b)/(TP_b+FN_b)
    b_f1 = (2*b_precision*b_recall)/(b_precision+b_recall)
    print(b_f1)

### 3.2 Creating CNN Models Using Keras & Tensorflow

In [ ]:
# Creating Bush CNN structure which resulted in optimal F1

CNN_model_bush = Sequential()

CNN_model_bush.add(Conv2D(128, kernel_size=2, activation="tanh", input_shape=(64,64,1))) 
CNN_model_bush.add(MaxPooling2D(pool_size=2))

CNN_model_bush.add(Conv2D(64, kernel_size=4, activation="tanh")) 
CNN_model_bush.add(MaxPooling2D(pool_size=4)) 

CNN_model_bush.add(Flatten()) 
CNN_model_bush.add(Dense(1, activation="sigmoid"))

CNN_model_bush.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Creating Williams CNN structure which resulted in optimal F1

CNN_model_williams = Sequential()

CNN_model_williams.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(64,64,1)))
CNN_model_williams.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

CNN_model_williams.add(Flatten())
CNN_model_williams.add(Dense(1, activation='sigmoid'))

CNN_model_williams.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### 3.3 Training CNN Model To Bush, Williams Data

In [ ]:
# Bush
CNN_model_bush.fit(X_train_bush, y_train_bush, validation_data=(X_test_bush, y_test_bush), epochs=5)

In [ ]:
# Williams
CNN_model_williams.fit(X_train_williams, y_train_williams, validation_data=(X_test_williams, y_test_williams), epochs=4)

In [ ]:
# Note: Saved versions of these models are available in the saved folder

### 3.4 Calculating F1 on Bush and Williams CNNs

In [ ]:
# For Bush

# On Test
b_CNN_predicts_te = np.round(CNN_model_bush.predict(X_test_bush))
print(np.sum(b_CNN_predicts)) # Number of predict true
# On Train
b_CNN_predict_tr = np.round(CNN_model_bush.predict(X_train_bush))

In [ ]:
# On Test
calc_f1(b_CNN_predicts_te, y_test_bush)
# On Train
calc_f1(b_CNN_predict_tr, y_train_bush)

In [ ]:
# For Williams

# On Test
w_CNN_predicts_te = np.round(CNN_model_williams.predict(X_test_williams))
np.sum(w_CNN_predicts) # Number of predict true
# On Train
w_CNN_predicts_tr = np.round(CNN_model_williams.predict(X_train_williams))

In [ ]:
# On Test
calc_f1(w_CNN_predicts_te, y_test_williams)
# On Train
calc_f1(w_CNN_predict_tr, y_train_williams)

## 4. Transfer Learning (Phase 4)

### 4.1 Importing Dataset for Pre-Training

In [ ]:
# More imports
from PIL import Image
import glob
import numpy as np
import pickle
from keras.models import load_model

In [ ]:
# One-time importing of Yale Extended B Faces Dataset

X = []; y_18 = []; # y_22 = []
i = 0
folderslist = glob.glob("ExtendedYaleB/*/")

for i in range(0,len(folderslist)):
    for filename in glob.glob(folderslist[i]+'*.pgm'):
        # Open image
        im = Image.open(filename)
        # Crop image to square
        width, height = im.size   
        left = (width - 480)/2
        top = (height - 480)/2
        right = (width + 480)/2
        bottom = (height + 480)/2
        im = im.crop((left, top, right, bottom))
        # Resize image to 64x64
        im = im.resize((64,64), Image.ANTIALIAS)
        # Convert to array for pickling purposes 
        im = np.array(im)
        # Add to images list
        X.append(im)
        # If person 18, label in y_18
        if(folderslist[i] == 'ExtendedYaleB/yaleB18/'):
            y_18.append(1)
        else:
            y_18.append(0)
    # Increment folders counter
    i = i + 1

In [ ]:
# Dumping data to an easier to load format
pickle.dump([X,y_18], open("yale_data.pickle","wb"))

In [ ]:
# Loading Pickle Data 
tmp = pickle.load(open("yale_data.pickle","rb"), encoding='latin1')

X = np.asarray(tmp[0])
y_18 = np.asarray(tmp[1])

In [ ]:
# Person 18 is the training target
X_train_18, X_test_18, y_train_18, y_test_18 = train_test_split(
    X, y_18, test_size=1./3, random_state=4000, shuffle=True, stratify=y_18)

### 4.2 Defining, Training, Testing on the Yale Dataset

In [ ]:
# Defining Model

CNN_model_18 = Sequential()

CNN_model_18.add(Conv2D(128, kernel_size=2, activation="tanh", input_shape=(64, 64, 1))) 
CNN_model_18.add(MaxPooling2D(pool_size=2))

CNN_model_18.add(Conv2D(64, kernel_size=4, activation="tanh")) 
CNN_model_18.add(MaxPooling2D(pool_size=4))

CNN_model_18.add(Conv2D(128, kernel_size=2, activation="tanh")) 
CNN_model_18.add(MaxPooling2D(pool_size=2))

CNN_model_18.add(Flatten()) 

CNN_model_18.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Training Model
CNN_model_18.fit(X_train_18, y_train_18, validation_data=(X_test_18, y_test_18), epochs=5)

In [ ]:
# Testing Model
CNN_predicts = np.round(CNN_model_18.predict(X_test_18))
# On Test
calc_f1(CNN_predicts, y_test_18)

### 4.3 Using Transfer Model on Bush

In [ ]:
# After loading transfer model copy as "transfer_mod_bush"
# transfer_mod_bush = copy.deepcopy(CNN_model_18)
transfer_mod_bush.fit(X_train_bush, y_train_bush, validation_data=(X_test_bush, y_test_bush), epochs=10)

In [ ]:
# On Test
b_CNN_predicts_te = np.round(transfer_mod_bush.predict(X_test_bush))
# On Train
b_CNN_predicts_tr = np.round(transfer_mod_bush.predict(X_train_bush))

In [ ]:
# On Test
calc_f1(b_CNN_predicts_te, y_test_bush)
# On Train
calc_f1(b_CNN_predict_tr, y_train_bush)

### 4.4 Using Transfer Model on Williams

In [ ]:
# After loading transfer model copy as "transfer_mod_williams"
# transfer_mod_williams = copy.deepcopy(CNN_model_18)
transfer_mod_williams.fit(X_train_williams, y_train_williams, validation_data=(X_test_williams, y_test_williams), epochs=5)

In [ ]:
# On Test
w_CNN_predicts_te = np.round(transfer_mod_williams.predict(X_test_williams))
# On Train
w_CNN_predicts_tr = np.round(transfer_mod_williams.predict(X_train_williams))

In [ ]:
# On Test
calc_f1(b_CNN_predicts_te, y_test_williams)
# On Train
calc_f1(b_CNN_predict_tr, y_train_williams)